In [44]:
from datetime import date, time

import pandas as pd
import numpy as np

pd.options.display.max_columns = 100
pd.options.display.max_rows = 300
pd.options.display.max_colwidth = 50

### The following section has been developed to created a rule table for the spec metadata. This logic is tuned to the XL created by Ashish. If work continues in this drirection to capture spec metadata, this section of the code will be useful

In [43]:
df_md = pd.read_excel("../data/external/New SKUs and SKUs Meta Data v1.0.xlsx", 
                   sheet_name='SKU Tags Meta Data', header=1)

df_md.rename(columns={'Unnamed: 0':'Area', 'Unnamed: 1':'Scope'}, inplace=True)
df_md.drop([x for i,x in enumerate(df_md.columns.values) if x.startswith('Un')], inplace=True, axis=1)
df_md.fillna("UNK", inplace=True)

df_md.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/external/New SKUs and SKUs Meta Data v1.0.xlsx'

In [ ]:
m_lst = []

for idx in range(2):
    temp = df_md[['Area', 'Scope']].copy()
    temp['tag'] = df_md.iloc[:,idx*2 + 2]
    temp['value'] = df_md.iloc[:,idx*2 + 3]

    lst = []

    for i in temp.index:
        for v_cnt in range(len(temp.value[i].split(','))):
            l1 = [x for x in temp.iloc[i,0:3].values]
            l2 = [temp.value[i].split(',')[v_cnt].strip()]

            l1.extend((l2))

            if np.sum([x == 'UNK' for x in l1]) < 4:
                lst.append(l1)

        temp_df = pd.DataFrame(lst)
        temp_df.columns = ['Area', 'Scope', 'Tag', 'Value']
        
    m_lst.append(temp_df)

In [ ]:
f_df = pd.concat(m_lst, axis=0)
f_df.head()

### The following code has been created to load a Postgres table with a structured schema and will contain the SKU data.

In [ ]:
df_ext = pd.read_excel("../data/external/lightning bid master template v3 price data.xlsm", 
                   sheet_name='Price Data', header=1)

df_ext.drop([x for x in df_ext.columns if x.startswith('Unnamed: ')], axis=1, inplace=True)
df_ext.columns = [x.replace(' ', '_') for x in df_ext.columns]
df_ext.columns = [x.replace('(', '') for x in df_ext.columns]
df_ext.columns = [x.replace(')', '') for x in df_ext.columns]
df_ext.columns = [x.replace('#', '') for x in df_ext.columns]

df_ext.drop(np.where(df_ext.Category.isnull() == True)[0], axis=0, inplace=True)

df_ext.reset_index(inplace=True, drop=True)

df_ext.RFP_Notes = df_ext.RFP_Notes.astype('object')
df_ext.Catalogue_ID_ = df_ext.Catalogue_ID_.astype('object')

df_ext.head(2)

In [ ]:
##df_ext.apply(lambda x: np.sum(x.isnull()), axis=0)

In [ ]:
##df_ext.dtypes

In [ ]:
# Filling all Missing values
key_fields = ['Category', 'Sub_Category', 'Work_Type', 'RFP_SPEC', 'RFP_SPEC_Tags']

other_required_fields = ['RFP_Notes', 'Owner', 'Property_Manager', 'Geography', 'Manufacture',
       'SKU_', 'Supplier', 'Catalogue_ID_', 'Retail_Name', 'Brandline', 'Notes', 'URL']

tag_fields = ['Color', 'Style_A', 'Style_B', 'Material', 'Finish',
       'Quality', 'Defining_Size', 'Tailorbird_Catalogue', 'Takeoff_Codex', 'Unit']

numeric_fields = ['Price']

info_fields = ['Price_Notes', 'Price_Source_Notes', 'Archive_Name']

df_ext.loc[:,key_fields] = df_ext[key_fields].fillna('ALL', axis=1)
df_ext.loc[:,other_required_fields] = df_ext[other_required_fields].fillna('ANY',  axis=1)
df_ext.loc[:,tag_fields] = df_ext[tag_fields].fillna('DNA', axis=1)
df_ext.loc[:,numeric_fields] = df_ext[numeric_fields].fillna(0, axis=1)
df_ext.loc[:,info_fields] = df_ext[info_fields].fillna('MI', axis=1)

df_ext.head()

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('postgresql://localhost/TB')

In [ ]:
df_ext.to_sql('sku_data', con=engine, index=False, if_exists='replace')

### Bulding the Spec Metadata

In [ ]:
import json

with open("../data/external/sample.json", encoding='utf-8') as f:
    data = json.load(f)


In [ ]:
items = pd.json_normalize(data, record_path=['items'], meta=['category_name'])
items.head()

In [ ]:
items.shape

## Read the spec object and match it with the SKU table

In [ ]:
import json
from bson import ObjectId
from bson import json_util
from pprint import pprint

class JSONEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, ObjectId):
            return str(o)
        return json.JSONEncoder.default(self, o)

##JSONEncoder().encode(analytics)

### Loaded JSON files from confluence note into data/external and now loading w/o issues

In [ ]:
with open('../data/external/pinecone.json') as f:
    project_data = json.load(f)

In [ ]:
pprint(project_data)

In [ ]:
# with open("../data/external/Pinecone.json", encoding='utf-8') as f:
#     p_txt = f.read()

In [ ]:
##json.loads(json_util.dumps(p_txt))

In [ ]:
# with open("../data/external/pinecone.json") as f:
#     p_data = json.loads(json_util.dumps(f.read()))

In [ ]:
# p_data

In [ ]:
# pd.read_json(p_data)

## Develop code to load data into MongoDB

In [ ]:
from pymongo import MongoClient
from pprint import pprint

In [ ]:
client = MongoClient('mongodb://127.0.0.1:27017/')

### >>> Added in following cells 

In [48]:
# create db, then get projects collection (lazy evaluation, not created until next block where we make an insertion)
test_db = client.test_db
# serverStatusResult = db.command("serverStatus")
# pprint(serverStatusResult)
projects = test_db['projects']

In [ ]:
# returns id of document once inserted into collection
project_id = projects.insert_one(project_data).inserted_id
print(f'Project id for Pinecone project is: {project_id}')

In [49]:
print(f'Collections in our test database: {test_db.list_collection_names()}')

Collections in our test database: ['projects']


In [63]:
def show_doc_fields(fields):
    """
    Helper function to format output of document retrieved from MongoDB collection
    """
    return {field : 1 for field in fields}
fields = [
    '_id', 'city', 'metadata', 'name',
    'project_type', 'state',
    'user_id', 'location'
]
show_fields = show_doc_fields(fields)

In [64]:
pprint(projects.find_one({'_id': project_id}, show_fields))

{'_id': ObjectId('60918fb46a87970a2a2b098c'),
 'city': 'For Collins',
 'location': '2212 Vermont Drive',
 'metadata': {'budget': '10000',
              'end_date': 'Thu, 22 Apr 2021 00:00:00 GMT',
              'project_owner': 'Atlas Group',
              'property_url': 'https://www.liveatpinecone.com/',
              'start_date': 'Thu, 22 Apr 2021 00:00:00 GMT'},
 'name': 'Pinecone',
 'project_type': 'MULTI_FAMILY',
 'state': 'CO',
 'user_id': 'a60f2a75-9e0a-4302-8f98-6e49370443b3'}


### >>> End of added cells insertion into MongoDB example <<<


In [ ]:
db = client.admin
# Issue the serverStatus command and print the results
serverStatusResult=db.command("serverStatus")
pprint(serverStatusResult)

In [ ]:
db = client.tb